In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile

In [3]:
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > DESIRED_ACCURACY:
      print("Reached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2022-01-23 02:14:16--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 108.177.98.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2022-01-23 02:14:16 (189 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation=tf.nn.relu),
tf.keras.layers.MaxPool2D((2,2)),
tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
tf.keras.layers.MaxPool2D((2,2)),
tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
tf.keras.layers.MaxPool2D((2,2)),

tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation=tf.nn.relu),
tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001), loss = tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [5]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_data_dir = os.path.join('./tmp')

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=10
        )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [6]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
    train_generator,
       steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks]
      )
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 3s 234ms/step - loss: 1.3661 - accuracy: 0.5875
Epoch 2/15
8/8 [==============================] - 2s 229ms/step - loss: 0.5152 - accuracy: 0.7500
Epoch 3/15
8/8 [==============================] - 2s 224ms/step - loss: 0.4064 - accuracy: 0.8125
Epoch 4/15
8/8 [==============================] - 2s 225ms/step - loss: 0.1065 - accuracy: 0.9875
Epoch 5/15
8/8 [==============================] - 2s 229ms/step - loss: 0.3571 - accuracy: 0.8500
Epoch 6/15
8/8 [==============================] - 2s 223ms/step - loss: 0.1364 - accuracy: 0.9500
Epoch 7/15
8/8 [==============================] - 2s 225ms/step - loss: 0.0666 - accuracy: 0.9750
Epoch 8/15
8/8 [==============================] - 2s 224ms/step - loss: 0.0539 - accuracy: 0.9750
Epoch 9/15
8/8 [==============================] - 2s 221ms/step - loss: 0.2056 - accuracy: 0.9125
Epoch 10/15
8/8 [==============================] - 2s 223ms/step - loss: 0.0132 - accuracy: 1.0000
